# Running the Micom Workflow for the Binary Azotobacter - Synechococcus Model

In this notebook we utilize the package `micom` to generate a binary community model for 2 organisms of interest:
- `Azotobacter vinelandii`
- `Synechococcus elongatus`

This binary consortium allows us to gain insights into the exchanges between the 2 organisms and run FBA experiments.

First off we can import all necessary packages for this notebook.

In [1]:
import pandas as pd
import cobra

from micom import Community
from micom.workflows import build, grow, tradeoff, fix_medium,build_database
from micom import load_pickle
from micom.viz import plot_tradeoff, plot_exchanges_per_sample, plot_growth

import os
os.environ["GRB_LICENSE_FILE"]

'/Users/mcna892/Desktop/Projects/Digital_Twins/gurobi.lic'

## Setting Up the Model in MICOM

To begin, we need to import our genome-scale models into `micom`. We have these models saved as `.sbml` files as this form is accepted easily into programs such as `cobrapy` and `micom`.

### Building a Taxonomy

Step #1: Establish a Taxonomy that lists the out the taxonomy for our organisms of interest

In [2]:
Tax= pd.DataFrame(columns=['id','genus','species','reactions','metabolites','sample_id','abundance'])
Tax.loc[len(Tax.index)] = ['Synechococcus', 'Synechococcus','S. elongatus',851,769,'One',500]
Tax.loc[len(Tax.index)] = ['Azotobacter','Azotobacter','A. vinelandii',2469,2003,'One',500]
Tax

,id,genus,species,reactions,metabolites,sample_id,abundance
0,Synechococcus,Synechococcus,S. elongatus,851,769,One,500
1,Azotobacter,Azotobacter,A. vinelandii,2469,2003,One,500


This taxonomy file lists important information for `micom` down the road, such as the number of `reactions` and `metabolites` in the provided models.

### Building a Database

Step #2: Now we must construct a database for the `.sbml` models to be preprocessed and stored. This is done by supplying `micom` with a file which contains model path locations.

In [5]:
db = pd.read_csv('./man_av_se.csv')
db

,file,kingdom,phylum,class,order,family,genus,species
0,./iJB785.xml,bacteria,Cyanobacteria,Cyanophyceae,Synechococcales,Synechococcaceae,Synechococcus,S. elongatus
1,./azo_vine.xml,bacteria,Pseudomonadota,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Azotobacter,A. vinelandii


In [6]:
build_database(db,'./db_av_se')

/Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"

[15:02:56] WARNING  The following sinks or demands were forced in the model and have been relaxed since  ]8;id=398020;file:///Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/micom/util.py\util.py]8;;\:]8;id=460386;file:///Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/micom/util.py#64\64]8;;\
                    they would interfere with MICOM's growth rate estimation: EX_sucr_e                            

,file,kingdom,phylum,class,order,family,genus,species,id,summary_rank
genus,,,,,,,,,,
Azotobacter,Azotobacter.json,bacteria,Pseudomonadota,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Azotobacter,A. vinelandii,Azotobacter,genus
Synechococcus,Synechococcus.json,bacteria,Cyanobacteria,Cyanophyceae,Synechococcales,Synechococcaceae,Synechococcus,S. elongatus,Synechococcus,genus


In [13]:
db_path = './db_av_se' 

### Construct Manifest Object

Step #3: Now that we have the Taxonomy and Database constructed we can build our community model. This is done by using the `build()` method in `micom`.

__Note__:

Build Manifest object from Taxonomy DataFrame and the corresponding database directory

Skip this step if manifest has already been built and saved to "models" directory


__IMPORTANT__: Declare the Solver you would like to use for this Community model here:
- osqp (good for smaller models)
- gurobi
- glpk
- cplex
- scipy


In [14]:
manifest = build(Tax, out_folder="models_av_se", model_db=db_path, cutoff=0.0001, threads=10,solver='gurobi')
manifest

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"


,sample_id,abundance,file,found_taxa,total_taxa,found_fraction,found_abundance_fraction
0,One,500,One.pickle,2.0,2.0,1.0,1.0


## Running the Models with FBA

Now that we have the `manifest`, we can load the model as a `Community` object through `micom`. This will give us some functionality similar to that of `cobrapy`. This can be done with the `load_pickle()` method we imported above through `micom`.

In [15]:
community = load_pickle("models_av_se/One.pickle")
print(len(community.reactions))

Read LP format model from file /var/folders/1f/ksln774x1hd1pzfgsjgpxt7r0000gn/T/tmp_jxqj2f_.lp
Reading time = 0.03 seconds
: 3116 rows, 7325 columns, 28585 nonzeros
3662


### Exploring the Model Attributes

Our new variable `community` behaves very similarly to a standard `cobrapy` model. We can explore it's attributes in a similar way as well.

Things such as `reactions` and `metabolites`:

In [16]:
community.reactions

[<Reaction EX_gln__L_e__Synechococcus at 0x34fe313a0>,
 <Reaction EX_hco3_e__Synechococcus at 0x34fe314c0>,
 <Reaction EX_mn2_e__Synechococcus at 0x34fe315b0>,
 <Reaction EX_arg__L_e__Synechococcus at 0x34fe316a0>,
 <Reaction ADPT__Synechococcus at 0x34fe31790>,
 <Reaction O2tcx__Synechococcus at 0x34fe31940>,
 <Reaction AOXPBDC__Synechococcus at 0x34fe31a00>,
 <Reaction DNTPPA__Synechococcus at 0x34fe31c10>,
 <Reaction CBMD__Synechococcus at 0x34fe31dc0>,
 <Reaction BIOMASS_PIGMENTS__Synechococcus at 0x34fe3e160>,
 <Reaction H2Otu_syn__Synechococcus at 0x34fe443d0>,
 <Reaction DTMPK__Synechococcus at 0x34fe61d90>,
 <Reaction BIOMASS__1__Synechococcus at 0x34fe78ee0>,
 <Reaction ORNTA__Synechococcus at 0x34fe88340>,
 <Reaction ATPM__Synechococcus at 0x34fe88610>,
 <Reaction ACOATA__Synechococcus at 0x34fe88460>,
 <Reaction 5DOAN__Synechococcus at 0x34fe88b50>,
 <Reaction 3HAD160__Synechococcus at 0x34feca310>,
 <Reaction RNDR3__Synechococcus at 0x34fecaca0>,
 <Reaction Htex__Synechococ

In [17]:
community.metabolites

[<Metabolite gln__L_e__Synechococcus at 0x34fe313d0>,
 <Metabolite hco3_e__Synechococcus at 0x34fe314f0>,
 <Metabolite mn2_e__Synechococcus at 0x34fe315e0>,
 <Metabolite arg__L_e__Synechococcus at 0x34fe316d0>,
 <Metabolite ade_c__Synechococcus at 0x34fe317c0>,
 <Metabolite amp_c__Synechococcus at 0x34fe317f0>,
 <Metabolite ppi_c__Synechococcus at 0x34fe31820>,
 <Metabolite prpp_c__Synechococcus at 0x34fe31850>,
 <Metabolite o2_c__Synechococcus at 0x34fe31970>,
 <Metabolite o2_cx__Synechococcus at 0x34fe318e0>,
 <Metabolite 2a3pp_c__Synechococcus at 0x34fe319d0>,
 <Metabolite 3a2oxpp_c__Synechococcus at 0x34fe31a30>,
 <Metabolite co2_c__Synechococcus at 0x34fe31b20>,
 <Metabolite h_c__Synechococcus at 0x34fe31af0>,
 <Metabolite ahdt_c__Synechococcus at 0x34fe31ca0>,
 <Metabolite dhpmp_c__Synechococcus at 0x34fe31cd0>,
 <Metabolite h2o_c__Synechococcus at 0x34fe31d30>,
 <Metabolite cbm_c__Synechococcus at 0x34fe31e80>,
 <Metabolite nh4_c__Synechococcus at 0x34fe31f10>,
 <Metabolite bm_p

and importantly the `medium`

In [18]:
community.medium

{'EX_hco3_m': 1.99,
 'EX_mn2_m': 999999.0,
 'EX_mg2_m': 999999.0,
 'EX_ca2_m': 999999.0,
 'EX_nh4_m': 5.0,
 'EX_fe2_m': 0.05,
 'EX_cu2_m': 999999.0,
 'EX_k_m': 999999.0,
 'EX_h2o_m': 999999.0,
 'EX_o2_m': 999999.0,
 'EX_co2_m': 1.99,
 'EX_leu__L_m': 1000.0,
 'EX_cobalt2_m': 999999.0,
 'EX_no3_m': 1.76,
 'EX_zn2_m': 999999.0,
 'EX_fe3_m': 0.05,
 'EX_so4_m': 999999.0,
 'EX_mobd_m': 999999.0,
 'EX_ni2_m': 999999.0,
 'EX_na1_m': 999999.0,
 'EX_h_m': 999999.0,
 'EX_photon410_m': 1000.0,
 'EX_photon430_m': 1000.0,
 'EX_photon450_m': 1000.0,
 'EX_photon470_m': 1000.0,
 'EX_photon490_m': 1000.0,
 'EX_photon510_m': 1000.0,
 'EX_photon530_m': 1000.0,
 'EX_photon550_m': 1000.0,
 'EX_photon570_m': 1000.0,
 'EX_photon590_m': 1000.0,
 'EX_photon610_m': 1000.0,
 'EX_photon630_m': 1000.0,
 'EX_photon650_m': 1000.0,
 'EX_photon670_m': 1000.0,
 'EX_photon690_m': 1000.0,
 'EX_pi_m': 999999.0,
 'EX_cl_m': 999999.0,
 'EX_glc__D_m': 5.0,
 'EX_sel_m': 999999.0,
 'EX_tungs_m': 999999.0,
 'EX_slnt_m': 999999.0

This behavior mimics the medium in `cobrapy`, but combines both models mediums into 1

### Running Optimization

Now that we have the model loaded, we can run standard `FBA` methods using `optimize()`. Default optimize does not return any fluxes from the model, so we can set the `fluxes=True` when calling the method to return them.

In [19]:
result = community.optimize(fluxes=True)
result

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.5,0.275669,2469,2003
Synechococcus,0.5,1.412054,851,768
medium,NaN,NaN,342,342


We can see that both organisms have a non-zero growth rate and that the community growth is also non-zero. Let's check the fluxes.

In [22]:
result.fluxes.T.loc['EX_nh4_e']

compartment
Azotobacter       0.0
Synechococcus   -10.0
medium            NaN
Name: EX_nh4_e, dtype: float64

#### Changing parts of the medium to test it's effect on growth

Now that we can successfully optimize the community model, we can begin altering the models medium and seeing how it changes the (community) growth rate.

First, let's make a copy of the original medium so that we can restore it after making changes.

In [24]:
medium_bkp = community.medium

Now we can make changes to the medium. The following cell is meant to be re-run with making changes. It will first restore the medium to the original and them set 

In [29]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
medium_to_change = community.medium

#Add or subtract reactions
medium_to_change["EX_nh4_m"] = 0
#medium_to_change["EX_sucr_m"] = 0
#medium_to_change["EX_leu__L_m"] = 1

# Set the new medium as the model's medium
community.medium = medium_to_change
community.medium

{'EX_hco3_m': 1.99,
 'EX_mn2_m': 999999.0,
 'EX_mg2_m': 999999.0,
 'EX_ca2_m': 999999.0,
 'EX_fe2_m': 0.05,
 'EX_cu2_m': 999999.0,
 'EX_k_m': 999999.0,
 'EX_h2o_m': 999999.0,
 'EX_o2_m': 999999.0,
 'EX_co2_m': 1.99,
 'EX_leu__L_m': 1000.0,
 'EX_cobalt2_m': 999999.0,
 'EX_no3_m': 1.76,
 'EX_zn2_m': 999999.0,
 'EX_fe3_m': 0.05,
 'EX_so4_m': 999999.0,
 'EX_mobd_m': 999999.0,
 'EX_ni2_m': 999999.0,
 'EX_na1_m': 999999.0,
 'EX_h_m': 999999.0,
 'EX_photon410_m': 1000.0,
 'EX_photon430_m': 1000.0,
 'EX_photon450_m': 1000.0,
 'EX_photon470_m': 1000.0,
 'EX_photon490_m': 1000.0,
 'EX_photon510_m': 1000.0,
 'EX_photon530_m': 1000.0,
 'EX_photon550_m': 1000.0,
 'EX_photon570_m': 1000.0,
 'EX_photon590_m': 1000.0,
 'EX_photon610_m': 1000.0,
 'EX_photon630_m': 1000.0,
 'EX_photon650_m': 1000.0,
 'EX_photon670_m': 1000.0,
 'EX_photon690_m': 1000.0,
 'EX_pi_m': 999999.0,
 'EX_cl_m': 999999.0,
 'EX_glc__D_m': 5.0,
 'EX_sel_m': 999999.0,
 'EX_tungs_m': 999999.0,
 'EX_slnt_m': 999999.0}

Now that the medium is changed, we can rerun the model optimization.

In [30]:
result_altered_medium = community.optimize(fluxes=True)

In [32]:
result_altered_medium.fluxes.T.loc['EX_nh4_e']

compartment
Azotobacter      1.07232
Synechococcus   -1.07232
medium               NaN
Name: EX_nh4_e, dtype: float64

## Running the Models with MICOM Grow

An alternative to running standard community optimization with `optimize()`, we can also use a `micom.workflows` method called `grow()`. This simulates growth of the organism while also simulating potential tradeoffs (between prioritizing community vs. individual growth). This method does not require our previously constructed `community` object, but rather the `manifest` we added earlier.

A key difference here though, is that we need to create a `DataFrame` detailing the reaction, flux, and metabolite as the medium provided to the method.

### Building the Medium

In [38]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
grow_medium_to_change = community.medium

#Add or subtract reactions
#grow_medium_to_change["EX_glc__D_m"] = 0
#grow_medium_to_change["EX_sucr_m"] = 1
grow_medium_to_change["EX_nh4_m"] = 0


In [39]:
grow_medium = pd.Series(grow_medium_to_change).to_frame('flux').reset_index()
grow_medium = grow_medium.rename(columns={'index':'reaction'})
grow_medium

,reaction,flux
0,EX_hco3_m,1.99
1,EX_mn2_m,999999.00
2,EX_mg2_m,999999.00
3,EX_ca2_m,999999.00
4,EX_nh4_m,0.00
5,EX_fe2_m,0.05
6,EX_cu2_m,999999.00
7,EX_k_m,999999.00
8,EX_h2o_m,999999.00
9,EX_o2_m,999999.00


In [40]:
result_grow = grow(manifest, model_folder="models_av_se", medium=grow_medium, tradeoff=0.01, threads=2,presolve=True)

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/1f/ksln774x1hd1pzfgsjgpxt7r0000gn/T/tmp8nq38lme.lp
Reading time = 0.03 seconds
: 3116 rows, 7325 columns, 28585 nonzeros


In [41]:
result_grow.exchanges

,taxon,sample_id,tolerance,reaction,flux,abundance,metabolite,direction
12,Azotobacter,One,0.000001,EX_h2_e,0.078976,0.5,h2_e,export
14,Azotobacter,One,0.000001,EX_ade_e,0.000002,0.5,ade_e,export
21,Azotobacter,One,0.000001,EX_zn2_e,-0.000003,0.5,zn2_e,import
28,Azotobacter,One,0.000001,EX_nh4_e,0.207363,0.5,nh4_e,export
63,Azotobacter,One,0.000001,EX_o2_e,-1.197536,0.5,o2_e,import
73,Azotobacter,One,0.000001,EX_no3_e,-0.286904,0.5,no3_e,import
99,Azotobacter,One,0.000001,EX_ac_e,1.787752,0.5,ac_e,export
100,Azotobacter,One,0.000001,EX_ni2_e,-0.000002,0.5,ni2_e,import
121,Azotobacter,One,0.000001,EX_h2o_e,1.657742,0.5,h2o_e,export
124,Azotobacter,One,0.000001,EX_h_e,1.283990,0.5,h_e,export


In [42]:
result_grow.exchanges.to_csv('av_se_out.csv')